In [ ]:
import sys
!{sys.executable} -m pip install sagemaker-experiments
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install tensorflow_datasets
!apt-get install -y make

import os
import boto3
sess = boto3.Session()
account_id = sess.client('sts').get_caller_identity()["Account"]
os.environ['DATASETS_BUCKET'] = f'sagemaker-test-titanic-{sess.region_name}-{account_id}'
os.environ['DATASETS_KEY_PREFIX'] = 'titanic'
!make upload_datasets

In [ ]:
from sagemaker.tensorflow import TensorFlow
import sagemaker

estimator = TensorFlow(
    source_dir='/root/sagemaker-tensorflow-titanic',
    entry_point='training.py',
    model_dir='/opt/ml/model', # => SM_MODEL_DIR
    output_path=f's3://{os.environ["DATASETS_BUCKET"]}/artifacts',
    role=sagemaker.get_execution_role(),
    framework_version='2.2.0',
    py_version='py37',
    train_instance_count=2,
    train_instance_type='ml.g4dn.xlarge',
    distributions={
        'parameter_server': { 'enabled': True }
    },
    hyperparameters={
        'dropout': 0.2,
    },
    metric_definitions=[
        {'Name':'test:accuracy', 'Regex':'train accuracy: (.*?),'},
        {'Name':'test:loss', 'Regex':'train loss: (.*?);'}
    ],
    enable_sagemaker_metrics=True,
)

inputs = sagemaker.inputs.s3_input(f's3://{os.environ["DATASETS_BUCKET"]}/{os.environ["DATASETS_KEY_PREFIX"]}')

estimator.fit(inputs={'training': inputs})